In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/RUSTEYE/dataset_cnn_v2.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
EPOCHS = 80

In [3]:
train_dir = '/content/dataset_cnn_v2/train'
val_dir   = '/content/dataset_cnn_v2/valid'
test_dir = '/content/dataset_cnn_v2/test'

# Data augmentation cho train
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 945 images belonging to 4 classes.
Found 202 images belonging to 4 classes.
Found 205 images belonging to 4 classes.


In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

# Thêm các layer phía trên
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [5]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 25,689,988 (98.00 MB)

 Trainable params: 2,102,276 (8.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

In [8]:
# Train
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 37s 425ms/step - accuracy: 0.3372 - loss: 1.3690 - val_accuracy: 0.4703 - val_loss: 1.0890
Epoch 2/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 24s 236ms/step - accuracy: 0.4649 - loss: 1.1180 - val_accuracy: 0.5743 - val_loss: 0.9426
Epoch 3/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 15s 248ms/step - accuracy: 0.5121 - loss: 0.9914 - val_accuracy: 0.5842 - val_loss: 0.8890
Epoch 4/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 20s 237ms/step - accuracy: 0.5693 - loss: 0.9084 - val_accuracy: 0.5792 - val_loss: 0.8748
Epoch 5/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 238ms/step - accuracy: 0.5483 - loss: 0.9372 - val_accuracy: 0.5693 - val_loss: 0.8618
Epoch 6/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 13s 213ms/step - accuracy: 0.5971 - loss: 0.8846 - val_accuracy: 0.5248 - val_loss: 0.9737
Epoch 7/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 234ms/step - accuracy: 0.5611 - loss: 0.9115 - val_accuracy: 0.5891 - val_loss: 0.8201
Epoch 8/80
60/60 ━━━━━━━━━━━━━━━━━━━━ 13s 219ms/step - accuracy: 0.5805 - loss: 0.8741 - val_accu

In [9]:
# Load best model
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')

In [10]:
# Đánh giá trên test set
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Lấy nhãn class
class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Metrics
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Accuracy theo từng class
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 334ms/step - accuracy: 0.5603 - loss: 0.8287

Test Loss: 0.6954
Test Accuracy: 67.32%
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 474ms/step
Accuracy: 67.31707317073172
Precision: 66.88085018426816
Recall: 67.31707317073172
F1 Score: 66.83380216199173

Classification Report:
                precision    recall  f1-score   support

 bong-troc-son     0.5750    0.4510    0.5055        51
day-cap-ri-set     0.6610    0.6842    0.6724        57
        ri-set     0.5932    0.6731    0.6306        52
       vet-nut     0.8723    0.9111    0.8913        45

      accuracy                         0.6732       205
     macro avg     0.6754    0.6798    0.6750       205
  weighted avg     0.6688    0.6732    0.6683       205


Accuracy for each class:
bong-troc-son: 0.45
day-cap-ri-set: 0.68
ri-set: 0.67
vet-nut: 0.91
